In [2]:
import pandas as pd
import numpy as np
from common_variables import *

In [3]:
df_csv = pd.read_csv(full_time_series_path, usecols=['Date','High','Low'], 
    index_col=['Date'], parse_dates=['Date'])
df = df_csv[df_csv.index >= pd.to_datetime(start_date)]
df

,High,Low
Date,,
2010-01-01 00:00:00,1.61673,1.61659
2010-01-01 00:01:00,1.61670,1.61670
2010-01-01 00:02:00,1.61670,1.61665
2010-01-01 00:03:00,1.61670,1.61670
2010-01-01 00:04:00,1.61678,1.61665
...,...,...
2023-12-01 21:55:00,1.27100,1.27087
2023-12-01 21:56:00,1.27114,1.27093
2023-12-01 21:57:00,1.27114,1.27094


In [4]:
df["HLAvg"] = df['High'].add(df['Low']).div(2)
del df['High']
del df['Low']
# Simple Moving Average
df['MA'] = df['HLAvg'].rolling(window=ma_periods).mean()
# Log Returns
df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
df.head(15)

,HLAvg,MA,Returns
Date,,,
2010-01-01 00:00:00,1.616660,NaN,NaN
2010-01-01 00:01:00,1.616700,NaN,NaN
2010-01-01 00:02:00,1.616675,NaN,NaN
2010-01-01 00:03:00,1.616700,NaN,NaN
2010-01-01 00:04:00,1.616715,NaN,NaN
2010-01-01 00:05:00,1.616675,NaN,NaN
2010-01-01 00:06:00,1.616725,NaN,NaN
2010-01-01 00:07:00,1.616725,NaN,NaN
2010-01-01 00:08:00,1.616725,NaN,NaN


In [5]:
df.dropna(how='any', inplace=True)
df = df[df.shape[0] % batch_size:]
df

,HLAvg,MA,Returns
Date,,,
2010-01-01 00:39:00,1.616600,1.616659,-2.209139e-06
2010-01-01 00:40:00,1.616650,1.616659,-4.418283e-07
2010-01-01 00:41:00,1.616650,1.616661,1.767312e-06
2010-01-01 00:42:00,1.616675,1.616663,1.104569e-06
2010-01-01 00:43:00,1.616700,1.616666,1.767307e-06
...,...,...,...
2023-12-01 21:55:00,1.270935,1.270822,1.405166e-06
2023-12-01 21:56:00,1.271035,1.270829,5.901676e-06
2023-12-01 21:57:00,1.271040,1.270834,3.934431e-06


In [6]:
df_train = df[:- validation_size - test_size]
df_validation = df[- validation_size - test_size - window_size:- test_size]
df_test = df[- test_size - window_size:]
print(f'df_train.shape {df_train.shape}, df_validation.shape {df_validation.shape}, df_test.shape {df_test.shape}')

df_train.shape (4676864, 3), df_validation.shape (262400, 3), df_test.shape (262400, 3)


In [7]:
df_train.to_csv(train_time_series_path)
df_validation.to_csv(validate_time_series_path)
df_test.to_csv(test_time_series_path)